In [1]:
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import heapq
# from sklearn.metrics import confusion_matrix
# # from sklearn.metrics import precision_recall_curve
# from sklearn.metrics import precision_score
# from sklearn.metrics import average_precision_score
# from sklearn.metrics import recall_score
from sklearn.metrics import f1_score 
# from sklearn.metrics import roc_curve
# from sklearn.metrics import auc
# from skimage.filters import threshold_otsu
# similarity_case_1_p = '../../../results/RF/simliarity_measure/case_1_ref_mask_similarity_measure.tif'
# similarity_case_2_p = '../../../results/RF/simliarity_measure/case_2_ref_mask_similarity_measure.tif'
# similarity_case_3_p = '../../../results/RF/simliarity_measure/case_3_ref_mask_similarity_measure.tif'
binary_change_map = '../../../results/RF/binary_change_D/change_map_case_1.tif'
similarity_change_map = '../../../results/RF/simliarity_measure/optimal_threshold/sim-change_map_case_1.tif'
source_proba_distr = '../../../results/RF/simliarity_measure/2018_rf_model_1.npy'
target_proba_distri = '../../../results/RF/simliarity_measure/2019_rf_model_1.npy'

gt_source_path  = '../../../data/rasterized_samples/2018_rasterizedImage.tif'
gt_target_path = '../../../data/rasterized_samples/2019_rasterizedImage.tif'

In [2]:
source_proba = np.load(source_proba_distr)

In [ ]:
source_proba_maxs = np.apply_along_axis(lambda x: heapq.nlargest(2, x), axis=1, arr = source_proba)
# import heapq


In [ ]:
uncertainty = source_proba_maxs[:,0] - source_proba_maxs[:,1]

In [ ]:
def changeErrorCheck(
    change_map,
    case,
    gt_source,
    gt_target,
    pred_source,
    pred_target, 
    outdir):

    """
    Base on the error in the change map and confusion matrix, the No Change to Change category is inspected to know the source of error.

    Input: raster files (change map, rasterized reference image(source and target), predicted image(source and target))

    Output: csv table with the error source (error occurences combination) and summarized error count.
        gt_source | gt_target | predicted_ source | predicted_target
    Note: Source and Target in the case are 2018 and 2019 respectively
    """

    # read rasters
    change_map_raster = rasterio.open(change_map).read(1)
    gt_source_raster = rasterio.open(gt_source).read(1)
    gt_target_raster = rasterio.open(gt_target).read(1)
    pred_source_raster = rasterio.open(pred_source).read(1)
    pred_target_raster = rasterio.open(pred_target).read(1)
    
    # from change_map_raster, return  where value == 2 
    change_mask = np.ma.masked_array(change_map_raster, mask=True)

    # write a function to optimize the above code
    def mask_and_extract(array, mask_array, mask_value):
        array = np.ma.masked_array(array, mask=True)
        array.mask[mask_array.data == mask_value] = False
        array_value = np.ma.compressed(array) #array_value = array.compressed()
        return array_value

    gt_source_ = mask_and_extract(gt_source_raster, change_mask, 2)
    gt_target_ = mask_and_extract(gt_target_raster, change_mask, 2)
    pred_source_ = mask_and_extract(pred_source_raster, change_mask, 2)
    pred_target_ = mask_and_extract(pred_target_raster, change_mask, 2)
